#### Create Dataset

In [29]:
import torch
import torchvision
import os
from torchvision import transforms
from os import path, mkdir
from PIL import Image, ImageOps
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split


convert_tensor = transforms.ToTensor()
images = 6400 #Amount of images you want to generate. Can be modified
background = 125
char  = 3
count = 0
min_char_size = 50
max_char_size = 150
dataset = []
for i in range(images):
    bg_rand = np.random.randint(1, background + 1)
    bgString = '../data/background/nature ({}) (Copy).jpg'.format(bg_rand)
    #char_rand = np.random.randint(0, char)
    char_rand = 0
    #size_rand = np.random.randint(min_char_size, max_char_size)
    size_rand = 100
    if i%2 == 0:
        charString = '../data/waldo/char{}.png'.format(char_rand)
        #label = torch.tensor([1, 0], dtype=torch.int32)
        label = 1
        #rand_x = 0
        #rand_y = 0
    else:
        charString = '../data/cat/notchar{}.png'.format(char_rand)
        #label = torch.tensor([0, 1], dtype=torch.int32)
        label = 0
        #rand_x = 150
        #rand_y = 150
    bg = Image.open(bgString).convert("RGBA")
    bg_x, bg_y = bg.size
    im = Image.open(charString)
    im = im.resize((size_rand,size_rand))
    #rand_pert = np.random.randint(0, 4)
    rand_pert = np.random.randint(5, 10)
    if rand_pert == 0:
        im = ImageOps.flip(im)
    elif rand_pert == 1:
        im = ImageOps.mirror(im)
    elif rand_pert == 2:
        im = ImageOps.flip(im)
        im = ImageOps.mirror(im)
    im_x, im_y = im.size
    rand_x = random.randint(0, bg_x-im_x)
    rand_y = random.randint(0, bg_y-im_y)

    bg.paste(im,(rand_x,rand_y),mask=im)
    bg = bg.convert("RGB")
    bg_tensor = convert_tensor(bg)
    dataset.append((bg_tensor,label))
    
    #Utkommenterad kod ifall man vill spara filerna lokalt
    #output_folder = 'test'
    #output_filename = 'test{}.png'.format(count)  # Specify the desired output file name with the .png extension
    #output_path = os.path.join(output_folder, output_filename)
    #if os.path.exists(output_path):
    #    os.chmod(output_path, 0o777)
    #bg.save(output_path, "PNG")
    count += 1
        
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image, label = self.data[index]
        return image, label
    
# Convert your dataset list to a CustomDataset object
custom_dataset = CustomDataset(dataset)

# Split the dataset into training and validation sets
train_ratio = 0.8  # Ratio of training set (0.8 = 80%)
train_size = int(train_ratio * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])

# Create a DataLoader using your custom dataset
batch_size = 64  # You can adjust this according to your needs
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#### ViT + main

In [30]:
import numpy as np
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
#from waldo import WaldoDataset
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm, trange
from torchvision.datasets.mnist import MNIST
from torch.utils.data import WeightedRandomSampler
from torchvision.models import resnet50
from matplotlib import pyplot as plt

#np.random.seed(0)
#torch.manual_seed(0)

from matplotlib import pyplot as plt

def patchify(images, n_patches):
    # Batch, Color Channels, Height, Width
    n, c, h, w = images.shape
    
    assert h == w
    
    # Batch, Number of Images, Size of each Patch
    patches = torch.zeros(n, n_patches ** 2, h * w * c // n_patches ** 2)
    patch_size = h // n_patches
    
    for idx, image in enumerate(images):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = image[:, i * patch_size: (i + 1) * patch_size, j * patch_size: (j+1)*patch_size]
                patches[idx, i * n_patches + j] = patch.flatten()

    return patches

def get_positional_embeddings(sequence_length, d):
    result = torch.ones(sequence_length, d)
    for i in range(sequence_length):
        for j in range(d):
            if j % 2 == 0:
                result[i][j] = np.sin(i / (10000 ** (j / d)))
            else:
                result[i][j] = np.cos(i / (10000 ** (j / d)))

    return result

class ViT(nn.Module):
    def __init__(self, colors, height, width, n_patches, hidden_dimension, n_heads, n_blocks, out_d = 1):
        super(ViT, self).__init__()

        self.c = colors
        self.h = height
        self.w = width
        
        self.n_patches = n_patches
        self.hidden_dimension = hidden_dimension
        
        assert self.h % self.n_patches == 0
        assert self.w % self.n_patches == 0
        self.patch_size = (self.h // n_patches, self.w // n_patches)
        
        # Linear mapper
        self.input_dimension = self.c * self.patch_size[0] * self.patch_size[1]
        self.linear1 = nn.Linear(self.input_dimension, self.hidden_dimension)
        
        # Classification token
        self.class_token = nn.Parameter(torch.rand(1, self.hidden_dimension))
        
        # Positional embedding
        self.register_buffer("pos_embed", get_positional_embeddings(self.n_patches ** 2 + 1, self.hidden_dimension), persistent=False)
        
        # Transformer encoder blocks
        self.blocks = nn.ModuleList([ViTBlock(self.hidden_dimension, n_heads) for _ in range(n_blocks)])
        
        # Classification MLP
        self.mlp = nn.Sequential(
            nn.Linear(self.hidden_dimension, out_d),
            # nn.Softmax(dim=-1)
        )
    
    def forward(self, images):
        n, c, h, w = images.shape
        patches = patchify(images, self.n_patches).to(self.pos_embed.device)
        tokens = self.linear1(patches)
        
        tokens = torch.cat((self.class_token.expand(n, 1, -1), tokens), dim=1)
        
        out = tokens + self.pos_embed.repeat(n, 1, 1)
        for block in self.blocks:
            out = block(out)
        
        out = out[:, 0]
        
        #out = torch.sigmoid(out)

        return self.mlp(out)
    
    
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d, n_heads=2):
        super(MultiHeadSelfAttention, self).__init__()
        self.d = d
        self.n_heads = n_heads
        
        assert d % n_heads == 0
        
        d_head = d // n_heads
        self.d_head = d_head
        self.q = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.k = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.v = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, sequences):
        result = []
        for sequence in sequences:
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q[head]
                k_mapping = self.k[head]
                v_mapping = self.v[head]
                
                seq = sequence[:, head * self.d_head: (head+1) * self.d_head]
                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)
                
                attention = self.softmax(q @ k.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v)
            result.append(torch.hstack(seq_result))
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])

class ViTBlock(nn.Module):
    def __init__(self, hidden_d, n_heads, mlp_ratio=4):
        super(ViTBlock, self).__init__()
        self.hidden_d = hidden_d
        self.n_heads = n_heads

        self.norm1 = nn.LayerNorm(hidden_d)
        self.mhsa = MultiHeadSelfAttention(hidden_d, n_heads)
        self.norm2 = nn.LayerNorm(hidden_d)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_d, mlp_ratio*hidden_d),
            nn.GELU(),
            nn.Linear(mlp_ratio * hidden_d, hidden_d)
        )
    
    def forward(self, x):
        out = x + self.mhsa(self.norm1(x))
        out = out + self.mlp(self.norm2(out))
        return out

if __name__ == "__main__":
    device = torch.device("cpu")#torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    
    
    
    
    #Ändra parametrar här
    model = ViT(3, 256, 256, 16, 128, 4, 3, 1).to(device) #colors, height, width, n_patches, hidden_dimension, n_heads, n_blocks, out_d = 1):
    #Ändra parametrar här

    EPOCHS = 10
    LR = 0.001
    transform = ToTensor()
    optimiser = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = torch.nn.MSELoss()
    #criterion = nn.BCELoss() 
    #criterion = nn.BCEWithLogitsLoss()
    #criterion = torch.nn.CrossEntropyLoss()
    for epoch in trange(EPOCHS, desc="Training"):
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} in training", leave=False):
            x, y = batch
            x, y = x.to(device), torch.unsqueeze(y.to(device), -1).to(torch.float32)
            y_hat = model(x)
            loss = criterion(y_hat, y)
            train_loss += loss.detach().cpu().item() / len(train_loader)
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
        
        print(f"Epoch {epoch + 1}/{EPOCHS} loss: {train_loss:.12f}")
    model.eval()
    correct, total = 0, 0
    test_loss = 0.0
    for batch in tqdm(val_loader, desc=f"Epoch {epoch + 1} in training", leave=False):
        x, y = batch
        x, y = x.to(device), torch.unsqueeze(y.to(device), -1).to(torch.float32)
        y_hat = model(x)
        loss = criterion(y_hat, y)
        test_loss += loss.detach().cpu().item() / len(train_loader)
        total += len(x)
        correct += (torch.round(y_hat) == y).detach().cpu().sum()
    print(f"Test loss: {test_loss}")
    print(f"Test Accuracy: {correct / total:.12f}  {correct} {total}")

Using device: cpu


Training:  10%|█         | 1/10 [02:39<23:55, 159.45s/it]           

Epoch 1/10 loss: 0.788814337552



Training:  20%|██        | 2/10 [05:23<21:35, 161.91s/it]           

Epoch 2/10 loss: 0.254609843716


...


Training:  90%|█████████ | 9/10 [23:55<02:39, 159.24s/it]           

Epoch 9/10 loss: 0.252800729498



Training: 100%|██████████| 10/10 [26:33<00:00, 159.34s/it]           


Epoch 10/10 loss: 0.250990558602


Test loss: 0.06205231714993715
Test Accuracy: 0.550000011921  704 1280
